Python执行过程：  

入口为 Programs/python.c
```
int
main(int argc, char **argv)
{
    // unix平台是_Py_UnixMain，windows平台是Py_Main
    return _Py_UnixMain(argc, argv);
}
```

然后为Modules/main.c
```
int
_Py_UnixMain(int argc, char **argv)
{
    _PyMain pymain = _PyMain_INIT;
    pymain.use_bytes_argv = 1;
    pymain.argc = argc;
    pymain.bytes_argv = argv;

    return pymain_main(&pymain);
}
```

```
static int
pymain_main(_PyMain *pymain)
{
    // 初始化
    pymain_init(pymain);

    int res = pymain_cmdline(pymain);
    if (res < 0) {
        _Py_FatalInitError(pymain->err);
    }
    if (res == 1) {
        goto done;
    }

    pymain_init_stdio(pymain);

    pymain->err = _Py_InitializeCore(&pymain->config);
    if (_Py_INIT_FAILED(pymain->err)) {
        _Py_FatalInitError(pymain->err);
    }

    if (pymain_init_python_main(pymain) < 0) {
        _Py_FatalInitError(pymain->err);
    }

    if (pymain_init_sys_path(pymain) < 0) {
        _Py_FatalInitError(pymain->err);
    }

    pymain_run_python(pymain);

    if (Py_FinalizeEx() < 0) {
        pymain->status = 120;
    }

done:
    // 退出清理
    pymain_free(pymain);

    return pymain->status;
}
```

```
static void
pymain_run_python(_PyMain *pymain)
{
    PyCompilerFlags cf = {.cf_flags = 0};

    pymain_header(pymain);
    pymain_import_readline(pymain);
    
    // 执行模式选择
    if (pymain->command) {
        // 命令行模式 -c
        pymain->status = pymain_run_command(pymain->command, &cf);
    }
    else if (pymain->module) {
        // 模块模式 -m
        pymain->status = (pymain_run_module(pymain->module, 1) != 0);
    }
    else {
        // 入口文件模式
        pymain_run_filename(pymain, &cf);
    }

    pymain_repl(pymain, &cf);
}
```


初始化，除内置类型外，最重要的是创建buildins、sys模块，并初始化sys.modules，sys.path等运行所需的环境配置。  
Python/pylifecycle.c  
```

```